# Bag of words

Tokenize.

In [1]:
from nltk.tokenize import TreebankWordTokenizer
sentence = "The faster Harry got to the store, the faster Harry, the faster, would get home."
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(sentence.lower())
tokens

['the',
 'faster',
 'harry',
 'got',
 'to',
 'the',
 'store',
 ',',
 'the',
 'faster',
 'harry',
 ',',
 'the',
 'faster',
 ',',
 'would',
 'get',
 'home',
 '.']

Create bag-of-words.

In [2]:
from collections import Counter
bag_of_words = Counter(tokens)
bag_of_words

Counter({'the': 4,
         'faster': 3,
         'harry': 2,
         'got': 1,
         'to': 1,
         'store': 1,
         ',': 3,
         'would': 1,
         'get': 1,
         'home': 1,
         '.': 1})

Explore the bag-of-words.

In [3]:
bag_of_words.most_common(4)

[('the', 4), ('faster', 3), (',', 3), ('harry', 2)]

In [7]:
time_harry_appears = bag_of_words['harry']
num_unique_words = len(bag_of_words)

print('The word "harry" appears {} times in the sentence.'.format(time_harry_appears))
print('The number of unique words in the sentence is {}.'.format(num_unique_words))

tf = time_harry_appears / num_unique_words
print('The term frequency of the word "harry" is {}.'.format(round(tf, 4)))

The word "harry" appears 2 times in the sentence.
The number of unique words in the sentence is 11.
The term frequency of the word "harry" is 0.1818.


The frequency of the term is way more important that the raw count. 

In [10]:
from collections import Counter
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

from nlpia.data.loaders import kite_text
tokens = tokenizer.tokenize(kite_text.lower())
token_counts = Counter(tokens)
token_counts.most_common(5)

[('the', 26), ('a', 20), ('kite', 16), (',', 15), ('and', 10)]

As we can see, there are a lot of stop words. Let's discard them.

In [11]:
import nltk
nltk.download('stopwords', quiet=True)
stopwords = nltk.corpus.stopwords.words('english')

tokens = [x for x in tokens if x not in stopwords]
kite_counts = Counter(tokens)
kite_counts.most_common(10)

[('kite', 16),
 (',', 15),
 ('kites', 8),
 ('wing', 5),
 ('lift', 4),
 ('may', 4),
 ('also', 3),
 ('kiting', 3),
 ('flown', 3),
 ('tethered', 2)]

From looking at this BOW, we can tell that this document has something about kite flying. Things get a bit more interesting when there are more documents.

# Vectorizing

In [13]:
document_vector = []
doc_length = len(tokens)
for key, value in kite_counts.most_common():
    document_vector.append(value / doc_length)
document_vector[:10]

[0.07207207207207207,
 0.06756756756756757,
 0.036036036036036036,
 0.02252252252252252,
 0.018018018018018018,
 0.018018018018018018,
 0.013513513513513514,
 0.013513513513513514,
 0.013513513513513514,
 0.009009009009009009]

In [17]:
docs = [
    "The faster Harry got to the store, the faster and faster Harry would get home.",
    "Harry is hairy and faster than Jill.",
    "Jill is not as hairy as Harry."
]
doc_tokens = []
for doc in docs:
    tokens = tokenizer.tokenize(doc.lower())
    print(tokens)
    doc_tokens += [sorted(tokens)]

for i, doc in enumerate(doc_tokens):
    print("doc{}: {}".format(i, len(doc)))

['the', 'faster', 'harry', 'got', 'to', 'the', 'store', ',', 'the', 'faster', 'and', 'faster', 'harry', 'would', 'get', 'home', '.']
['harry', 'is', 'hairy', 'and', 'faster', 'than', 'jill', '.']
['jill', 'is', 'not', 'as', 'hairy', 'as', 'harry', '.']
doc0: 17
doc1: 8
doc2: 8


In [18]:
all_doc_tokens = sum(doc_tokens, [])
len(all_doc_tokens)

33

In [19]:
lexicon = sorted(set(all_doc_tokens))
print(len(lexicon))
print(lexicon)

18
[',', '.', 'and', 'as', 'faster', 'get', 'got', 'hairy', 'harry', 'home', 'is', 'jill', 'not', 'store', 'than', 'the', 'to', 'would']


In [20]:
from collections import OrderedDict
zero_vector = OrderedDict((token, 0) for token in lexicon)
zero_vector

OrderedDict([(',', 0),
             ('.', 0),
             ('and', 0),
             ('as', 0),
             ('faster', 0),
             ('get', 0),
             ('got', 0),
             ('hairy', 0),
             ('harry', 0),
             ('home', 0),
             ('is', 0),
             ('jill', 0),
             ('not', 0),
             ('store', 0),
             ('than', 0),
             ('the', 0),
             ('to', 0),
             ('would', 0)])

In [21]:
import copy
doc_vectors = []
for doc in docs:
    vec = copy.copy(zero_vector)
    tokens = tokenizer.tokenize(doc.lower())
    token_counts = Counter(tokens)
    
    for key, value in token_counts.items():
        vec[key] = value / len(lexicon)
    doc_vectors.append(vec)
doc_vectors

[OrderedDict([(',', 0.05555555555555555),
              ('.', 0.05555555555555555),
              ('and', 0.05555555555555555),
              ('as', 0),
              ('faster', 0.16666666666666666),
              ('get', 0.05555555555555555),
              ('got', 0.05555555555555555),
              ('hairy', 0),
              ('harry', 0.1111111111111111),
              ('home', 0.05555555555555555),
              ('is', 0),
              ('jill', 0),
              ('not', 0),
              ('store', 0.05555555555555555),
              ('than', 0),
              ('the', 0.16666666666666666),
              ('to', 0.05555555555555555),
              ('would', 0.05555555555555555)]),
 OrderedDict([(',', 0),
              ('.', 0.05555555555555555),
              ('and', 0.05555555555555555),
              ('as', 0),
              ('faster', 0.05555555555555555),
              ('get', 0),
              ('got', 0),
              ('hairy', 0.05555555555555555),
              ('harry', 0.05

## Vector spaces

In [22]:
import math
def consine_sim(vec1, vec2):
    vec1 = [val for val in vec1.values()]
    vec2 = [val for val in vec2.values()]
    dot_prod = 0
    for i, v in enumerate(vec1):
        dot_prod += v * vec2[i]
    mag_1 = math.sqrt(sum([x**2 for x in vec1]))
    mag_2 = math.sqrt(sum([x**2 for x in vec2]))
    return dot_prod / (mag_1 * mag_2)

## Zipf's law

In [23]:
nltk.download('brown')
from nltk.corpus import brown
brown.words()[:10]

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\lived\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of']

In [24]:
brown.tagged_words()[:10]

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN')]

In [25]:
from collections import Counter
puncs = set((',', '.', '--', '-', '!', '?', ':', ';', '``', "''", '(', ')', '[', ']'))
word_list = [word.lower() for word in brown.words() if word not in puncs]
word_counts = Counter(word_list)
word_counts.most_common(10)

[('the', 69971),
 ('of', 36412),
 ('and', 28853),
 ('to', 26158),
 ('a', 23195),
 ('in', 21337),
 ('that', 10594),
 ('is', 10109),
 ('was', 9815),
 ('he', 9548)]

The most common word ("the") is **twice** as frequent as the next common words ("of", "and"), **three times** as common as the next one ("a", "in"), and **six times** as common as "that", "is", "was", and "he".

# Topic modeling
When there are multiple documents, words with low frequency against the lexicon can be important in the document where it belongs to. To counter this, we need **inverse document frequency**.

In [26]:
from nlpia.data.loaders import kite_text, kite_history
kite_intro = kite_text.lower()
intro_tokens = tokenizer.tokenize(kite_intro)

kite_history = kite_history.lower()
history_tokens = tokenizer.tokenize(kite_history)

intro_total = len(intro_tokens)
print('Number of tokens in intro:', intro_total)

history_total = len(history_tokens)
print('Number of tokens in history:', history_total)

Number of tokens in intro: 363
Number of tokens in history: 297


In [27]:
intro_tf = {}
history_tf = {}

intro_counts  = Counter(intro_tokens)
history_counts = Counter(history_tokens)

intro_tf['kite'] = intro_counts['kite'] / intro_total
history_tf['kite'] = history_counts['kite'] / history_total

print('Term Frequency of "kite" in intro is:', round(intro_tf['kite'], 4))
print('Term Frequency of "kite" in history is:', round(history_tf['kite'], 4))

Term Frequency of "kite" in intro is: 0.0441
Term Frequency of "kite" in history is: 0.0202


The word "kite" appears more often in the intro than the history. But the question is, is it really more important in the intro?

In [28]:
intro_tf['and'] = intro_counts['and'] / intro_total
history_tf['and'] = history_counts['and'] / history_total

print('Term Frequency of "and" in intro is:', round(intro_tf['and'], 4))
print('Term Frequency of "and" in history is:', round(history_tf['and'], 4))

Term Frequency of "and" in intro is: 0.0275
Term Frequency of "and" in history is: 0.0303


One way to think about IDF is this: if a term appears a lot in a document, but rarely in others, it is probably important for that document.

In [30]:
num_docs_containing_and = 0
for doc in [intro_tokens, history_tokens]:
    if 'and' in doc:
        num_docs_containing_and += 1
        
intro_tf['china'] = intro_counts['china'] / intro_total
history_tf['china'] = history_counts['china'] / history_total

num_docs = len([intro_tokens, history_tokens])
intro_idf = {}
history_idf = {}

intro_idf['and'] = num_docs / num_docs_containing_and
history_idf['and'] = num_docs / num_docs_containing_and


intro_idf['kite'] = num_docs / 2
history_idf['kite'] = num_docs / 2

intro_idf['china'] = num_docs / 1
history_idf['china'] = num_docs / 1

intro_tfidf = {}
intro_tfidf['and'] = intro_tf['and'] * intro_idf['and']
intro_tfidf['kite'] = intro_tf['kite'] * intro_idf['kite']
intro_tfidf['china'] = intro_tf['china'] * intro_idf['china']

history_tfidf = {}
history_tfidf['and'] = history_tf['and'] * history_idf['and']
history_tfidf['kite'] = history_tf['kite'] * history_idf['kite']
history_tfidf['china'] = history_tf['china'] * history_idf['china']

print('TF-IDF for "and" in intro is:', round(intro_tfidf['and'], 4))
print('TF-IDF for "and" in history is:', round(history_tfidf['and'], 4))
print()

print('TF-IDF for "kite" in intro is:', round(intro_tfidf['kite'], 4))
print('TF-IDF for "kite" in history is:', round(history_tfidf['kite'], 4))
print()

print('TF-IDF for "china" in intro is:', round(intro_tfidf['china'], 4))
print('TF-IDF for "china" in history is:', round(history_tfidf['china'], 4))

TF-IDF for "and" in intro is: 0.0275
TF-IDF for "and" in history is: 0.0303

TF-IDF for "kite" in intro is: 0.0441
TF-IDF for "kite" in history is: 0.0202

TF-IDF for "china" in intro is: 0.0
TF-IDF for "china" in history is: 0.0202


## Relevance ranking

In [31]:
document_tfidf_vectors = []
for doc in docs:
    vec = copy.copy(zero_vector)
    tokens = tokenizer.tokenize(doc.lower())
    token_counts = Counter(tokens)
    
    for term, count in token_counts.items():
        docs_containing_term = 0
        for _doc in docs:
            if term in _doc:
                docs_containing_term += 1
        tf = count / len(lexicon)
        if docs_containing_term:
            idf = len(docs) / docs_containing_term
        else:
            idf = 0
        vec[term] = tf * idf
    document_tfidf_vectors.append(vec)

In [34]:
for vec in document_tfidf_vectors:
    print(vec)

OrderedDict([(',', 0.16666666666666666), ('.', 0.05555555555555555), ('and', 0.08333333333333333), ('as', 0), ('faster', 0.25), ('get', 0.16666666666666666), ('got', 0.16666666666666666), ('hairy', 0), ('harry', 0.0), ('home', 0.16666666666666666), ('is', 0), ('jill', 0), ('not', 0), ('store', 0.16666666666666666), ('than', 0), ('the', 0.5), ('to', 0.16666666666666666), ('would', 0.16666666666666666)])
OrderedDict([(',', 0), ('.', 0.05555555555555555), ('and', 0.08333333333333333), ('as', 0), ('faster', 0.08333333333333333), ('get', 0), ('got', 0), ('hairy', 0.08333333333333333), ('harry', 0.0), ('home', 0), ('is', 0.08333333333333333), ('jill', 0.0), ('not', 0), ('store', 0), ('than', 0.16666666666666666), ('the', 0), ('to', 0), ('would', 0)])
OrderedDict([(',', 0), ('.', 0.05555555555555555), ('and', 0), ('as', 0.1111111111111111), ('faster', 0), ('get', 0), ('got', 0), ('hairy', 0.08333333333333333), ('harry', 0.0), ('home', 0), ('is', 0.08333333333333333), ('jill', 0.0), ('not', 0.

Having built the TF-IDF vectors of the documents, we now use cosine similarity to find similar documents given a query.

In [35]:
query = "How long does it take to get to the store?"
query_vec = copy.copy(zero_vector)

tokens = tokenizer.tokenize(query.lower())
token_counts = Counter(tokens)

for term, count in token_counts.items():
    docs_containing_term = 0
    for _doc in docs:
        if term in _doc:
            docs_containing_term += 1
    if docs_containing_term == 0:
        continue
    tf = count / len(tokens)
    idf = len(docs) / docs_containing_term
    query_vec[term] = tf * idf
print(query_vec)

OrderedDict([(',', 0), ('.', 0), ('and', 0), ('as', 0), ('faster', 0), ('get', 0.2727272727272727), ('got', 0), ('hairy', 0), ('harry', 0), ('home', 0), ('is', 0), ('jill', 0), ('not', 0), ('store', 0.2727272727272727), ('than', 0), ('the', 0.2727272727272727), ('to', 0.5454545454545454), ('would', 0)])


In [36]:
for vec in document_tfidf_vectors:
    print(consine_sim(query_vec, vec))

0.6132857433407973
0.0
0.0


We can safely say that the first document is most similar to our query.

In [38]:
print('Query: ', query)
print('Most similar document: ', docs[0])

Query:  How long does it take to get to the store?
Most similar document:  The faster Harry got to the store, the faster and faster Harry would get home.


## Tools
We can use `scikit-learn` to implement the above model.

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = docs

vectorizer = TfidfVectorizer(min_df=1)
model = vectorizer.fit_transform(corpus)
tfidf_vecs = model.todense().round(2)
print(tfidf_vecs)

[[0.16 0.   0.48 0.21 0.21 0.   0.25 0.21 0.   0.   0.   0.21 0.   0.64
  0.21 0.21]
 [0.37 0.   0.37 0.   0.   0.37 0.29 0.   0.37 0.37 0.   0.   0.49 0.
  0.   0.  ]
 [0.   0.75 0.   0.   0.   0.29 0.22 0.   0.29 0.29 0.38 0.   0.   0.
  0.   0.  ]]


In [52]:
query_vec_tfidf = vectorizer.transform([query])
query_vec_tfidf.todense().round(2)

array([[0.  , 0.  , 0.  , 0.38, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.38, 0.  , 0.38, 0.76, 0.  ]])

In [53]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(query_vec_tfidf, tfidf_vecs)

array([[0.56144043, 0.        , 0.        ]])